In [0]:
# Tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis
import torch
from torchtext import data

In [0]:
# Manual Seed for reproducibility
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 164k/84.1M [00:00<00:56, 1.48MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 67.0MB/s]


In [4]:
len(train_data), len(test_data)

(25000, 25000)

In [6]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))
len(train_data), len(valid_data), len(test_data)

(17500, 7500, 25000)

In [8]:
# Build Vocabulary
VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=VOCAB_SIZE)
LABEL.build_vocab(train_data)

len(TEXT.vocab), len(LABEL.vocab)

(25002, 2)

In [0]:
# Create Iterators
BATCH_SIZE = 64

device = torch.device(
            'cuda' if torch.cuda.is_available()
            else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
            (train_data, valid_data, test_data),
            batch_size=BATCH_SIZE,
            device=device)

# Model
Three layers:
- Embeddings 
- RNN
- Linear

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, emebedding_dim, hidden_dim, output_dim):
        super().__init__()

        # Var name = nn.Layer(input dimension, output dimension)
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text):
        output, hidden = self.rnn(self.embedding(text))

        assert torch.equal(output[-1, :, :], hidden.squeeze(0))

        return self.fc(hidden.squeeze(0))

In [0]:
input_dim = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1
model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

In [22]:
print(model)
print("Trainable param:", sum(p.numel() for p in model.parameters() if p.requires_grad))

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)
Trainable param 2592105


# Defining Train and Eval function

In [0]:
import torch.optim as optim
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
# Accuracy function
## Sigmoid function and threshold
def accuracy(y_pred, y_orig):
    # Threshold of 0.5
    y_pred = torch.round(torch.sigmoid(y_pred))
    correct = (y_pred == y_orig).float()
    accuracy = correct.sum() / len(correct)
    return accuracy


In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    # Put model in training mode which turns dropouts and batch normalization
    model.train()
    for data in iterator:
        # Have to manually zero optimizer gardients
        optimizer.zero_grad()

        # Forward pass
        y_pred = model(data.text).squeeze(1)
        loss = criterion(y_pred, data.label)
        acc = accuracy(y_pred, data.label)

        # Backward pass: comp gradients
        loss.backward()
        # Update weights 
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item()
        epoch_acc += loss.item()
    
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    # Put model in evaluation mode: turn off dropout and normalization
    model.eval()

    with torch.no_grad():
        for data in iterator:
            y_pred = model(data.text).squeeze(1)
            loss = criterion(y_pred, data.label)
            acc = accuracy(y_pred, data.label)

            epoch_loss += loss.item()
            epoch_acc += loss.item()
    
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Train

In [44]:
epochs = 5

best_val_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    e_min, e_sec = epoch_time(start_time, end_time)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'senti.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {e_min}m {e_sec}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\tVal Loss: {val_loss:.3f} | Val acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train acc: 69.34%
	Val Loss: 0.698 | Val acc: 69.78%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train acc: 69.32%
	Val Loss: 0.698 | Val acc: 69.80%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train acc: 69.33%
	Val Loss: 0.698 | Val acc: 69.77%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train acc: 69.33%
	Val Loss: 0.698 | Val acc: 69.79%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train acc: 69.32%
	Val Loss: 0.698 | Val acc: 69.80%


# Test

In [45]:
model.load_state_dict(torch.load('senti.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'\tTest Loss: {test_loss:.3f} | Test acc: {test_acc*100:.2f}%')

	Test Loss: 0.712 | Test acc: 71.17%
